# Question 4：高维度矩阵处理
## 二进制读取，压缩格式，追赶法，高斯消去
### Part1 
第一步先用两个小维度的数据来
- 熟悉一下矩阵的数据格式
- 读入矩阵数据，并存储数据矩阵
- 求解的简单编写  
- 有毒，老师那里给出了另一种读取数据的方式。
--- 
仍然剩下的问题：
是要通过一个函数将4个同事求解完，还是设置一个通用的函数模型？
我先写通用的参数原型把

In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import struct 

### 读取数据
**集成**一下压缩格式(514)和未压缩格式(258)的区别

In [6]:
# f = open('data20191.dat','rb')
# # 读取文件的标识头
# Header = []
# for i in range(5):
#     buff = f.read(4)
#     data = struct.unpack('l',buff)
# #     print(data[0])
#     Header.append(data)
#     pass
# # 输出文件的标识头
# print('--------------header-----------------')
# print(Header)

# # 构建初始系数矩阵形式
# temp = np.array(Header)
# temp = temp.reshape((-1))
# n = temp[2]
# A = np.zeros((n,n))
# b = np.zeros(n)

# # 先分开些，在用if 控制
# # 初始化循环参数
# RowVa = 0
# ColVa = 0

# # 循环装入系数
# buff = f.read(4)
# while buff:
#     data = struct.unpack('f',buff)
#     if RowVa<n : A[RowVa,ColVa] = data[0]
#     else : b[ColVa] = data[0]
#     ColVa += 1
#     buff = f.read(4)
#     if ColVa == 10:
#         ColVa = 0
#         RowVa +=1
#         pass
#     pass

# print('--------------A-----------------')
# print(A)
# print('--------------B-----------------')
# print(b)
# f.close()

In [5]:
# # 这是压缩格式的矩阵读取的形式，但是我觉得最终可能并不需要读取矩阵，只需要他的下标把？
# f = open('data20192.dat','rb')
# # 读取文件的标识头
# Header = []
# for i in range(5):
#     buff = f.read(4)
#     data = struct.unpack('l',buff)
# #     print(data[0])
#     Header.append(data)
#     pass
# # 输出文件的标识头
# print('--------------header-----------------')
# print(Header)

# # 构建初始系数矩阵形式
# temp = np.array(Header)
# temp = temp.reshape((-1))
# n = temp[2]
# p = temp[3]
# q = temp[4]
# A = np.zeros((n,n))
# b = np.zeros(n)
# # 初始化循环参数
# RowVa = 0
# ColVa = 0
# offset = 0
# buff = f.read(4)
# while buff:
#     data = struct.unpack('f',buff)
#     if RowVa<n:
#         A[RowVa,ColVa+offset] = data[0]
#     else :
#         print('FUCk')
#         for i in range(n):
#             data = struct.unpack('f',buff)
#             b[i] = data[0]
#             buff = f.read(4)
#             pass
        
        
#     buff = f.read(4)
#     ColVa += 1 
#     if ColVa == p+q+1:
#         ColVa = 0
#         RowVa += 1 
#         if (RowVa > q and (RowVa+q)<n):
#             offset += 1
            
# print('--------------A-----------------')
# print(A)
# print('--------------B-----------------')
# print(b)
# f.close()

### 接着要完成的是求解部分：
- 高斯消去法的消去函数编写（正向简化函数）
- 数值代入计算和回代过程（根据高斯消去法的简化矩阵反向回代计算过程）
- 输出最终结果

In [66]:
f = open('data20191.dat','rb')
# 读取文件的标识头
Header = []
for i in range(5):
    buff = f.read(4)
    data = struct.unpack('l',buff)
#     print(data[0])
    Header.append(data)
    pass
# 输出文件的标识头
print('--------------header for array-----------------')
print(Header)

# 构建初始系数矩阵形式
# 收集输出系数矩阵的必要参数
temp = np.array(Header)
temp = temp.reshape((-1))
Version = temp[1]
n = temp[2]
p = temp[3]
q = temp[4]
A = np.zeros((n,n))
b = np.zeros(n)

# 初始化循环参数：对应行列的index 以及offset
# 
RowVa = 0
ColVa = 0
offset = 0

# 版本控制主体循环框架：258非压缩格式、514压缩格式
if Version == 258:
    # 通过buff控制读取文件的光标位置，根据给出的格式，每四个位是一个数字
    # while控制将文件读取完毕
    buff = f.read(4)
    while buff:
        # 用unpack函数解析float存储的2进制结构
        data = struct.unpack('f',buff)
        # 分开稀疏矩阵A和右端向量b的读取
        if RowVa<n : A[RowVa,ColVa] = data[0]
        else : b[ColVa] = data[0]
        # 以下控制光标和矩阵位置的移动
        ColVa += 1
        buff = f.read(4)
        if ColVa == 10:
            ColVa = 0
            RowVa +=1
            pass
        pass
    # 输出结果
    print('--------------A for the 258-----------------')
    print(A)
    print('--------------B for the 258-----------------')
    print(b)
elif Version == 514:
    buff = f.read(4)
    while buff:
        data = struct.unpack('f',buff)
        if RowVa<n:
            A[RowVa,ColVa+offset] = data[0]
        else :
            # 直接一次将b全部读完
            
            for i in range(n):
                data = struct.unpack('f',buff)
                b[i] = data[0]
                buff = f.read(4)
                pass
            
        buff = f.read(4)
        # 控制光标和矩阵位置的移动
        # 在压缩格式之下，着重对矩阵所属位置的偏移进行控制
        ColVa += 1 
        if ColVa == p+q+1:
            ColVa = 0
            RowVa += 1 
            # 主要基于存储格式的话，考虑下带宽对矩阵数据位置的影像
            # 在左端不在移动的顶点和右侧不在一栋的顶点加以控制。
            if (RowVa > q and (RowVa+q)<n):
                offset += 1
                
    print('--------------A for the 514-----------------')
    print(A)
    print('--------------B for the 514-----------------')
    print(b)

f.close()

--------------header for array-----------------
[(-236858976,), (258,), (10,), (3,), (3,)]
--------------A for the 258-----------------
[[ 13.76444435   3.17777777   2.31999993   4.76666689   0.
    0.           0.           0.           0.           0.        ]
 [  0.75999999  17.91333389   5.30000019  -1.91999996   4.4333334
    0.           0.           0.           0.           0.        ]
 [  4.92000008   0.82499999  19.09500122  -0.30000001  -2.95000005
    0.60000002   0.           0.           0.           0.        ]
 [  3.70000005   2.37142849   3.57500005  27.68976212   1.65999997
    2.4333334    3.45000005   0.           0.           0.        ]
 [  0.           3.5999999    1.73333335   1.51111114  17.84222221
    0.93333334   1.84444439   1.72000003   0.           0.        ]
 [  0.           0.           4.76666689   0.62222224  -0.06666667
   34.15555573  -2.9000001    7.5333333   -3.76666665   0.        ]
 [  0.           0.           0.           0.64999998  -1.28888

In [82]:
filename = 'data201801.dat'
Headinfo = np.fromfile(filename,dtype=np.int32)
head = Headinfo[0:5].astype(np.int32)
print(head)
print('``````````````````````head↑```````````````````````````')
Version = head[1]
n = head[2]
p = head[3]
q = head[4]
offset =[]
for i in range(n):
    if i>q:offset.append(i-q)
    else: offset.append(0)
offset = np.array(offset)
data = np.fromfile(filename,dtype=np.float32)
# 针对不同格式的矩阵进行不同类型的读取
# 压缩格式的带状矩阵的0值在后续消去的时候进行判断后填充
if Version == 258:
    start = 5
    end = n*n+5
    A = data[start:end].astype(np.float32)
    b = data[end:end+n].astype(np.float32)
    A = A.reshape((n,n))
    pass
elif Version == 514:
    start = 5
    end = n*(p+q+1)+5
    A = data[start:end].astype(np.float32)
    b = data[end:end+n].astype(np.float32)
    num = p+q+1
    A = A.reshape((n,num))

#     print(A)
#     print('``````````````````````originA```````````````````````````')
    for i in range(p):
        index = -1
        # 找到第一个非0 元所在的index
        for j in range(num):
            if A[i][j] !=0 and index == -1: 
                index = j
        # 重构矩阵进行替换
        if index != -1:
            for k in range(num-index):
                A[i][k] = A[i][k+index]
            A[i][num-index:num] = 0
print(A)
print('````````````````````reshpae A↑`````````````````````````````')
print(b)
print('````````````````````reshape b↑`````````````````````````````')

# 接下来编写高斯消去法（带状矩阵专用）
# 消去的过程中只需要有限的取值就行

def GaussEli(A,b,Version,n,q,p,offset):
    if Version == 258: offset[:]=0
    for index in range(n):
        for i in range(1,q+1):
            if index+i<n:
                l = A[index+i,index-offset[index+i]]/A[index,index-offset[index]]
                for j in range(p+1):
                    if index+j<n:
                        A[index+i,index+j-offset[index+i]] -= A[index,index+j-offset[index]]*l
                        A[index+i,index+j-offset[index+i]] = A[index+i,index+j-offset[index+i]]
                b[index+i] -= b[index]*l
    # 上面得到了A，b消去以后的矩阵，后面通过回带求解x[]
    x = np.zeros(n)
    for i in range(n-1,-1,-1):
        sumA = 0
        for j in range(p):
            index = i+j
            if index<n: sumA += A[i,index-offset[i]]*x[index]
            
        for j in range(q):
            index = i-j
            if index>0: sumA += A[i,index-offset[i]]*x[index]

        x[i] = (b[i]-sumA)/A[i,i-offset[i]]
    print(x)
    print('`````````````````````result↑````````````````````````````')
    print(A)
    print('`````````````````````gauss a↑````````````````````````````')
    print(b)
    print('`````````````````````gauss b↑````````````````````````````')
    print(offset)
    print('``````````````````````offset↑```````````````````````````')

    pass

GaussEli(A,b,Version,n,q,p,offset)

[-236858976        258         10          3          3]
``````````````````````head↑```````````````````````````
[[11.457144    0.6571429   4.4         1.9         0.          0.
   0.          0.          0.          0.        ]
 [ 2.7666667  22.466667   -1.7666667  -6.5333333   0.9         0.
   0.          0.          0.          0.        ]
 [-4.8         4.72       32.05611     3.325      -1.8444444   9.866667
   0.          0.          0.          0.        ]
 [ 5.12       -0.72       -4.2285714  19.867142    2.15        3.2285714
  -2.92        0.          0.          0.        ]
 [ 0.          1.32        1.8        -0.51111114 14.05611    -2.575
   0.95        3.4         0.          0.        ]
 [ 0.          0.          0.52       -4.866667    6.5333333  27.92
  -4.9333334   4.866667    0.7         0.        ]
 [ 0.          0.          0.         11.8         2.3        -2.1
  35.793335    3.5333333  -2.36        2.2       ]
 [ 0.          0.          0.          0.         

In [10]:
# for i in range(5,-1,-1):print(i)

In [13]:
from scipy.linalg import solve
x = solve(A,b)
print(x)

[2.019236  2.0177734 2.0183036 ... 2.0179572 2.0196075 2.02161  ]
